In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [3]:
flight_arrivals = pd.read_csv('Arriving Data.csv')
flight_departures = pd.read_csv('Departing Data.csv')

In [4]:
flight_arrivals.drop('NewMsgTime',axis=1,inplace=True)
flight_departures.drop('NewMsgTime',axis=1,inplace=True)
flight_arrivals.head()

,FltNbr,AircraftType,SchedDepTime,SchedArrTime,BlockArrDate,BlockArrTime,MsgDate,MsgTime,NewMsgDate,BlockDepDate,BlockDepTime,TimeToFly,DepCountry,Error
0,4,321,715,830,231112,838,231112,722,231112,231112,721,60,6,16
1,4,321,715,830,231112,838,231112,733,231112,231112,721,50,6,15
2,4,321,715,830,231112,838,231112,756,231112,231112,721,32,6,10
3,4,321,715,830,231112,838,231112,817,231112,231112,721,19,6,2
4,4,321,715,830,231112,838,231112,824,231112,231112,721,14,6,0


In [5]:
flight_arrivals.shape

(280332, 14)

In [6]:
mean = flight_arrivals.mean(axis=0)
meanE = flight_arrivals['Error'].mean(axis=0)
stdE = flight_arrivals['Error'].std(axis=0)
flight_arrivals -= mean
std = flight_arrivals.std(axis=0)
flight_arrivals /= std

ValueError: Cannot broadcast operands together.

In [ ]:
X = flight_arrivals.drop('Error',axis=1)
y = flight_arrivals['Error']

In [ ]:
x = X.as_matrix()
y = y.as_matrix()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
test_df = pd.DataFrame(y_test,columns=['True Error'])
test_df.head()

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from sklearn.metrics import r2_score

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                          input_shape=(x_train.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
k=4
num_val_samples = len(x_train) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = x_test[i * num_val_samples: (i + 1) * num_val_samples]
    partial_x_train = np.concatenate(
        [x_train[:i * num_val_samples],
        x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_y_train = np.concatenate(
        [y_train[:i * num_val_samples],
        y_train[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_x_train, partial_y_train,
            epochs=num_epochs, batch_size=32, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


"""x_val = x_train[:80000]
partial_x_train = x_train[80000:]

y_val = y_train[:80000]
partial_y_train = y_train[80000:]"""

In [ ]:
num_epochs = 150
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partial_x_train = np.concatenate(
        [x_train[:i * num_val_samples],
        x_train[(i + 1) * num_val_samples:]],
        axis=0)

    partial_y_train = np.concatenate(
        [y_train[:i * num_val_samples],
        y_train[(i + 1) * num_val_samples:]],
        axis=0)

    model = build_model()
    history = model.fit(partial_x_train, partial_y_train,
                validation_data=(val_data, val_targets),
                epochs=num_epochs, batch_size=32, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)
    


In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
"""model = build_model()
history = model.fit(partial_x_train, partial_y_train,
                   validation_data=(x_val, y_val),
                   epochs=150, batch_size=128, verbose=0)""""""

In [ ]:
"""val_mae = history.history['val_mean_absolute_error']"""

In [ ]:
"""loss = history.history['loss']
val_loss = history.history['val_loss']"""

In [ ]:
"""epochs = range(1, len(loss) + 1)"""

In [ ]:
"""plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()"""

In [ ]:
"""mae_history = history.history['val_mean_absolute_error']

plt.plot(range(1, len(mae_history) + 1), mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')

plt.show()"""

In [ ]:
arr_model = models.Sequential()
arr_model.add(layers.Dense(64, activation='relu',
                      input_shape=(x_train.shape[1],)))
arr_model.add(layers.Dense(64, activation='relu'))
arr_model.add(layers.Dense(1))
arr_model.compile(optimizer = 'SGD', loss='mse', metrics=['mae'])

arr_model.fit(x_train, y_train,
         epochs=75, batch_size=128, verbose=0)
results = model.evaluate(x_test, y_test)

In [ ]:
results[0]

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predicted_df = pd.DataFrame(predictions,columns=['Predicted Error'])

In [ ]:
# stand_feat = pd.DataFrame(x_test,columns=flight_data.columns[:-1])

In [ ]:
unstand_predict = (predicted_df * stdE) + meanE

In [ ]:
unstand_predict.head()

In [ ]:
r2_score(y_test,predictions)

In [ ]:
mean = flight_departures.mean(axis=0)
meanE = flight_departures['Error'].mean(axis=0)
stdE = flight_departures['Error'].std(axis=0)
flight_departures -= mean
std = flight_departures.std(axis=0)
flight_departures /= std

X = flight_departures.drop('Error',axis=1)
y = flight_departures['Error']
x = X.as_matrix()
y = y.as_matrix()

x_train_dep, x_test_dep, y_train_dep, y_test_dep = train_test_split(x, y, test_size=0.3)

In [ ]:
flight_departures.shape

In [ ]:
x_val_dep = x_train_dep[:20000]
partial_x_train_dep = x_train_dep[20000:]

y_val_dep = y_train_dep[:20000]
partial_y_train_dep = y_train_dep[20000:]

In [ ]:
model = models.Sequential()
model.add(layers.Dense(128, activation='relu',
                      input_shape=(x_train_dep.shape[1],)))
model.add(layers.Dense(1))
model.compile(optimizer = 'SGD', loss='mse', metrics=['mae'])

history = model.fit(partial_x_train_dep, partial_y_train_dep,
                   validation_data=(x_val_dep, y_val_dep),
                   epochs=80, batch_size=128, verbose=0)
val_mae = history.history['val_mean_absolute_error']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

In [ ]:
x=4
while True:
    i = 2**x
    dep_model = models.Sequential()
    dep_model.add(layers.Dense(128, activation='relu',
                          input_shape=(x_train_dep.shape[1],)))
    dep_model.add(layers.Dense(i, activation='relu'))
    dep_model.add(layers.Dense(1))
    dep_model.compile(optimizer = 'SGD', loss='mse', metrics=['mae'])
    dep_model.fit(x_train_dep, y_train_dep,
             epochs=75, batch_size=128, verbose=0)
    results = dep_model.evaluate(x_test_dep, y_test_dep)
    predictions = dep_model.predict(x_test_dep)
    r2 = r2_score(y_test_dep,predictions)
    print('batch_size {}, r squared = {}'.format(i,r2))
    if r2 < 0.71 and x < 9:
        x += 1
    else:
        break

In [ ]:
results[0]

In [ ]:
predicted_df = pd.DataFrame(predictions,columns=['Predicted Error'])

In [ ]:
unstand_predict = (predicted_df * stdE) + meanE
unstand_predict.head()

In [ ]:
print(r2)

## 